# Arabic Tweets Sentiment Analysis with AraBERT Tokenizer and BYOM
Using pre-trained AraBERT model for tokenization and Catboost for sentiment classification inside Vantage

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import re
import os
import nltk
import getpass as gp
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import onnxruntime as ort
from tqdm import tqdm
import teradataml as tdml
import catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Download stopwords if not already downloaded
nltk.download('stopwords', quiet=True)
arabic_stopwords = stopwords.words('arabic')

## 2. Load and Preprocess Dataset

In [4]:
# Load dataset
df = pd.read_csv('Arabic Sentiment Analysis Dataset - SS2030.csv')
df = df.reset_index().rename(columns={'index': 'tid', 'text': 'docstring'})
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (4252, 3)


,tid,docstring,Sentiment
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,1
2,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,1
3,3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,1
4,4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,1


In [5]:
# Preprocessing function
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Remove non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in arabic_stopwords]
    return ' '.join(words)

# Clean the text
df['cleaned_doc'] = df['docstring'].apply(clean_text)
df = df[df['cleaned_doc'].notna() & (df['cleaned_doc'].astype(str).str.strip() != '')]
df = df[['tid', 'cleaned_doc', 'Sentiment']]
print(f"\nSentiment distribution:\n{df['Sentiment'].value_counts()}")
df.head()


Sentiment distribution:
Sentiment
1    2436
0    1815
Name: count, dtype: int64


,tid,cleaned_doc,Sentiment
0,0,حقوق المرأة,1
1,1,حقوق المرأة الإسلام,1
2,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1
3,3,حقوق المرأة تضمنها وزارة العدل,1
4,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1


## 3. Load AraBERT Classifier Model

In [ ]:
# Download model if needed (run only once)
# Uncomment to download:
model_name = "PRAli22/AraBert-Arabic-Sentiment-Analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer.save_pretrained("./arabert_sentiment_model")
model.save_pretrained("./arabert_sentiment_model")

In [7]:
# Load model from local directory
model_dir = "./arabert_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
model.eval()
print("Model loaded successfully")

Model loaded successfully


## 4. Generating Embeddings of the text using Arabert Tokenizer

In [8]:
# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    with torch.no_grad():
        outputs = model.bert(**inputs)  # Get BERT base outputs
        # Use [CLS] token embedding (first token)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy().flatten()
    return embeddings

In [9]:
# Generate embeddings for all texts
print("Generating embeddings...")
embeddings_list = []
for text in tqdm(df['cleaned_doc'], desc="Creating embeddings"):
    emb = get_embeddings(text)
    embeddings_list.append(emb)

Generating embeddings...


Creating embeddings: 100%|██████████| 4251/4251 [03:12<00:00, 22.08it/s]


In [10]:
# Convert list of embeddings to numpy array
embeddings_array = np.array(embeddings_list, dtype=np.float32)

# Create column names for embeddings (emb_0, emb_1, ..., emb_767)
embedding_columns = [f'emb_{i}' for i in range(embeddings_array.shape[1])]

# Create dataframe with embeddings as separate columns
embeddings_df = pd.DataFrame(embeddings_array, columns=embedding_columns, dtype=np.float32)

# Add tid, text, and sentiment columns
embeddings_df.insert(0, 'tid', df['tid'].values)
embeddings_df.insert(1, 'cleaned_doc', df['cleaned_doc'].values)
embeddings_df['Sentiment'] = df['Sentiment'].values

print(f"\nDataframe shape: {embeddings_df.shape}")
print(f"Number of embedding columns: {len(embedding_columns)}")
print(f"\nColumn names: {embeddings_df.columns.tolist()[:5]}... (first 5)")
print(f"\nData types:\n{embeddings_df.dtypes.head(10)}")
print("\nFirst few rows:")
embeddings_df.head()


Dataframe shape: (4251, 771)
Number of embedding columns: 768

Column names: ['tid', 'cleaned_doc', 'emb_0', 'emb_1', 'emb_2']... (first 5)

Data types:
tid              int64
cleaned_doc     object
emb_0          float32
emb_1          float32
emb_2          float32
emb_3          float32
emb_4          float32
emb_5          float32
emb_6          float32
emb_7          float32
dtype: object

First few rows:


,tid,cleaned_doc,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,Sentiment
0,0,حقوق المرأة,0.915905,-0.582944,1.188565,-1.250446,1.040004,-1.015085,-3.424124,-0.472961,...,-0.102470,-0.293598,-0.512675,-0.347759,-0.649128,0.602792,0.992201,-2.023002,0.048379,1
1,1,حقوق المرأة الإسلام,0.769661,-1.120466,0.702905,-1.390138,0.862794,-0.822431,-3.307889,0.168463,...,-0.492879,-0.406484,-0.717845,0.276217,0.050306,0.271503,1.274817,-1.587476,0.246192,1
2,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1.582610,1.252096,1.246331,-2.210566,-0.014998,0.320480,-2.154431,-1.159533,...,0.823797,0.896441,-0.089060,-0.165477,-0.669874,0.585659,0.447458,-0.787317,-0.296942,1
3,3,حقوق المرأة تضمنها وزارة العدل,0.835202,-1.044387,0.848940,-0.381577,0.344434,-1.078913,-3.103493,-0.616793,...,-0.593038,0.565501,-0.508824,0.036350,0.635917,-0.261125,0.594527,-1.912073,0.470904,1
4,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,0.394898,0.734406,0.526005,-1.466056,-0.266501,1.107806,-1.597542,-0.261610,...,-1.743913,0.732452,-2.575912,0.220516,0.800939,0.618779,0.600923,1.654606,-0.109317,1


## 5. BYOM (Tokenizer + Classifier) in Vantage

for this purpose, we will be using the tokenizer from Arabert coupled with logistic regression for binary classification. The tokenizer, transformer model and our own trained model (LR) will be stored in vantage for scoring at scale with the help of BYOM

Lets load the preprocessed data into Vantage

In [ ]:
td_context = tdml.create_context(host="host address added here",
                            username="username added here",
                            password = gp.getpass(prompt='Password:')
                           )

In [12]:
tdml.copy_to_sql(embeddings_df, table_name="arabic_sentiment_data", if_exists="replace", primary_index='tid')
tdf = tdml.DataFrame("arabic_sentiment_data")
tdf.head()

cleaned_doc     emb_0     emb_1     emb_2     emb_3     emb_4     emb_5     emb_6     emb_7     emb_8     emb_9    emb_10    emb_11    emb_12    emb_13    emb_14    emb_15    emb_16    emb_17    emb_18    emb_19    emb_20    emb_21    emb_22    emb_23    emb_24    emb_25    emb_26    emb_27    emb_28    emb_29    emb_30    emb_31    emb_32    emb_33    emb_34    emb_35    emb_36    emb_37    emb_38    emb_39    emb_40    emb_41    emb_42    emb_43    emb_44    emb_45    emb_46    emb_47    emb_48    emb_49    emb_50    emb_51    emb_52    emb_53    emb_54    emb_55    emb_56    emb_57    emb_58    emb_59    emb_60    emb_61    emb_62    emb_63    emb_64    emb_65    emb_66    emb_67    emb_68    emb_69    emb_70    emb_71    emb_72    emb_73    emb_74    emb_75    emb_76    emb_77    emb_78    emb_79    emb_80    emb_81    emb_82    emb_83    emb_84    emb_85    emb_86    emb_87    emb_88    emb_89    emb_90    emb_91    emb_92    emb_93    emb_94    emb_95    emb_96    emb_97    emb_98    emb_99   emb_100   emb_101   emb_102   emb_103   emb_104   emb_105   emb_106   emb_107   emb_108   emb_109   emb_110   emb_111   emb_112   emb_113   emb_114   emb_115   emb_116   emb_117   emb_118   emb_119   emb_120   emb_121   emb_122   emb_123   emb_124   emb_125   emb_126   emb_127   emb_128   emb_129   emb_130   emb_131   emb_132   emb_133   emb_134   emb_135   emb_136   emb_137   emb_138   emb_139   emb_140   emb_141   emb_142   emb_143   emb_144   emb_145   emb_146   emb_147   emb_148   emb_149   emb_150   emb_151   emb_152   emb_153   emb_154   emb_155   emb_156   emb_157   emb_158   emb_159   emb_160   emb_161   emb_162   emb_163   emb_164   emb_165   emb_166   emb_167   emb_168   emb_169   emb_170   emb_171   emb_172   emb_173   emb_174   emb_175   emb_176   emb_177   emb_178   emb_179   emb_180   emb_181   emb_182   emb_183   emb_184   emb_185   emb_186   emb_187   emb_188   emb_189   emb_190   emb_191   emb_192   emb_193   emb_194   emb_195   emb_196   emb_197   emb_198   emb_199   emb_200   emb_201   emb_202   emb_203   emb_204   emb_205   emb_206   emb_207   emb_208   emb_209   emb_210   emb_211   emb_212   emb_213   emb_214   emb_215   emb_216   emb_217   emb_218   emb_219   emb_220   emb_221   emb_222   emb_223   emb_224   emb_225   emb_226   emb_227   emb_228   emb_229   emb_230   emb_231   emb_232   emb_233   emb_234   emb_235   emb_236   emb_237   emb_238   emb_239   emb_240   emb_241   emb_242   emb_243   emb_244   emb_245   emb_246   emb_247   emb_248   emb_249   emb_250   emb_251   emb_252   emb_253   emb_254   emb_255   emb_256   emb_257   emb_258   emb_259   emb_260   emb_261   emb_262   emb_263   emb_264   emb_265   emb_266   emb_267   emb_268   emb_269   emb_270   emb_271   emb_272   emb_273   emb_274   emb_275   emb_276   emb_277   emb_278   emb_279   emb_280   emb_281   emb_282   emb_283   emb_284   emb_285   emb_286   emb_287   emb_288   emb_289   emb_290   emb_291   emb_292   emb_293   emb_294   emb_295   emb_296   emb_297   emb_298   emb_299   emb_300   emb_301   emb_302   emb_303   emb_304   emb_305   emb_306   emb_307   emb_308   emb_309   emb_310   emb_311   emb_312   emb_313   emb_314   emb_315   emb_316   emb_317   emb_318   emb_319   emb_320   emb_321   emb_322   emb_323   emb_324   emb_325   emb_326   emb_327   emb_328   emb_329   emb_330   emb_331   emb_332   emb_333   emb_334   emb_335   emb_336   emb_337   emb_338   emb_339   emb_340   emb_341   emb_342   emb_343   emb_344   emb_345   emb_346   emb_347   emb_348   emb_349   emb_350   emb_351   emb_352   emb_353   emb_354   emb_355   emb_356   emb_357   emb_358   emb_359   emb_360   emb_361   emb_362   emb_363   emb_364   emb_365   emb_366   emb_367   emb_368   emb_369   emb_370   emb_371   emb_372   emb_373   emb_374   emb_375   emb_376   emb_377   emb_378   emb_379   emb_380   emb_381   emb_382   emb_383   emb_384   emb_385   emb_386   emb_387   emb_388   emb_389   emb_390   emb_391   emb_392   emb_393   emb_394   emb_395   emb_396   emb_397   emb_39

Splitting the training and test dataset

In [13]:
TrainTestSplit_out = tdml.TrainTestSplit(
                                    data = tdf,
                                    id_column = "tid",
                                    train_size = 0.70,
                                    test_size = 0.30,
                                    seed = 21
)

# Split into 2 virtual dataframes
df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

In [14]:
df_train.head()

tid                                                                                                                                                                                                                                   cleaned_doc     emb_0     emb_1     emb_2     emb_3     emb_4     emb_5     emb_6     emb_7     emb_8     emb_9    emb_10    emb_11    emb_12    emb_13    emb_14    emb_15    emb_16    emb_17    emb_18    emb_19    emb_20    emb_21    emb_22    emb_23    emb_24    emb_25    emb_26    emb_27    emb_28    emb_29    emb_30    emb_31    emb_32    emb_33    emb_34    emb_35    emb_36    emb_37    emb_38    emb_39    emb_40    emb_41    emb_42    emb_43    emb_44    emb_45    emb_46    emb_47    emb_48    emb_49    emb_50    emb_51    emb_52    emb_53    emb_54    emb_55    emb_56    emb_57    emb_58    emb_59    emb_60    emb_61    emb_62    emb_63    emb_64    emb_65    emb_66    emb_67    emb_68    emb_69    emb_70    emb_71    emb_72    emb_73    emb_74    emb_75    emb_76    emb_77    emb_78    emb_79    emb_80    emb_81    emb_82    emb_83    emb_84    emb_85    emb_86    emb_87    emb_88    emb_89    emb_90    emb_91    emb_92    emb_93    emb_94    emb_95    emb_96    emb_97    emb_98    emb_99   emb_100   emb_101   emb_102   emb_103   emb_104   emb_105   emb_106   emb_107   emb_108   emb_109   emb_110   emb_111   emb_112   emb_113   emb_114   emb_115   emb_116   emb_117   emb_118   emb_119   emb_120   emb_121   emb_122   emb_123   emb_124   emb_125   emb_126   emb_127   emb_128   emb_129   emb_130   emb_131   emb_132   emb_133   emb_134   emb_135   emb_136   emb_137   emb_138   emb_139   emb_140   emb_141   emb_142   emb_143   emb_144   emb_145   emb_146   emb_147   emb_148   emb_149   emb_150   emb_151   emb_152   emb_153   emb_154   emb_155   emb_156   emb_157   emb_158   emb_159   emb_160   emb_161   emb_162   emb_163   emb_164   emb_165   emb_166   emb_167   emb_168   emb_169   emb_170   emb_171   emb_172   emb_173   emb_174   emb_175   emb_176   emb_177   emb_178   emb_179   emb_180   emb_181   emb_182   emb_183   emb_184   emb_185   emb_186   emb_187   emb_188   emb_189   emb_190   emb_191   emb_192   emb_193   emb_194   emb_195   emb_196   emb_197   emb_198   emb_199   emb_200   emb_201   emb_202   emb_203   emb_204   emb_205   emb_206   emb_207   emb_208   emb_209   emb_210   emb_211   emb_212   emb_213   emb_214   emb_215   emb_216   emb_217   emb_218   emb_219   emb_220   emb_221   emb_222   emb_223   emb_224   emb_225   emb_226   emb_227   emb_228   emb_229   emb_230   emb_231   emb_232   emb_233   emb_234   emb_235   emb_236   emb_237   emb_238   emb_239   emb_240   emb_241   emb_242   emb_243   emb_244   emb_245   emb_246   emb_247   emb_248   emb_249   emb_250   emb_251   emb_252   emb_253   emb_254   emb_255   emb_256   emb_257   emb_258   emb_259   emb_260   emb_261   emb_262   emb_263   emb_264   emb_265   emb_266   emb_267   emb_268   emb_269   emb_270   emb_271   emb_272   emb_273   emb_274   emb_275   emb_276   emb_277   emb_278   emb_279   emb_280   emb_281   emb_282   emb_283   emb_284   emb_285   emb_286   emb_287   emb_288   emb_289   emb_290   emb_291   emb_292   emb_293   emb_294   emb_295   emb_296   emb_297   emb_298   emb_299   emb_300   emb_301   emb_302   emb_303   emb_304   emb_305   emb_306   emb_307   emb_308   emb_309   emb_310   emb_311   emb_312   emb_313   emb_314   emb_315   emb_316   emb_317   emb_318   emb_319   emb_320   emb_321   emb_322   emb_323   emb_324   emb_325   emb_326   emb_327   emb_328   emb_329   emb_330   emb_331   emb_332   emb_333   emb_334   emb_335   emb_336   emb_337   emb_338   emb_339   emb_340   emb_341   emb_342   emb_343   emb_344   emb_345   emb_346   emb_347   emb_348   emb_349   emb_350   emb_351   emb_352   emb_353   emb_354   emb_355   emb_356   emb_357   emb_358   emb_359   emb_360   emb_361   emb_362   emb_363   emb_364   emb_365   emb_366   emb_367   emb_368   emb_369   emb_370   emb_371   emb_372   emb_373   emb_374   emb_37

Pulling out training set from Vantage to local disk

In [15]:
df_train.to_csv("trainset.csv", all_rows=True, fastexport=False)
training_dataset = pd.read_csv("trainset.csv")
training_dataset.head()


Data is successfully exported into trainset.csv


,tid,cleaned_doc,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,Sentiment
0,1,حقوق المرأة الإسلام,0.769661,-1.120466,0.702905,-1.390138,0.862794,-0.822431,-3.307889,0.168463,...,-0.492879,-0.406484,-0.717845,0.276217,0.050306,0.271503,1.274817,-1.587476,0.246192,1
1,9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق ا...,2.687572,-0.492569,0.918326,-1.587362,0.456493,0.878994,-1.847191,0.188100,...,-0.319201,0.755966,-0.639121,-0.377064,1.697193,-0.357151,0.048731,0.910484,-1.209859,1
2,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,0.394898,0.734406,0.526005,-1.466056,-0.266501,1.107806,-1.597542,-0.261610,...,-1.743913,0.732452,-2.575912,0.220516,0.800939,0.618779,0.600923,1.654606,-0.109317,1
3,0,حقوق المرأة,0.915905,-0.582944,1.188565,-1.250446,1.040004,-1.015085,-3.424124,-0.472961,...,-0.102470,-0.293598,-0.512675,-0.347759,-0.649128,0.602792,0.992201,-2.023002,0.048379,1
4,3,حقوق المرأة تضمنها وزارة العدل,0.835202,-1.044387,0.848940,-0.381577,0.344434,-1.078913,-3.103493,-0.616793,...,-0.593038,0.565501,-0.508824,0.036350,0.635917,-0.261125,0.594527,-1.912073,0.470904,1


In [16]:
X = training_dataset[[f"emb_{i}" for i in range(768)]].values
y = training_dataset["Sentiment"].values

In [17]:
#Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [18]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((2380, 768), (595, 768), (2380,), (595,))

Lets utilize Catboost for classification because why not!! (actually it is tree based so does not require scaling and requires minimal tuning for good performance (good job by Yandex))

In [19]:
model_catb = catboost.CatBoostClassifier(loss_function="CrossEntropy", iterations=1000, eval_metric="AUC", random_seed=42, verbose=100)
model_catb.fit(X_train, y_train, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	total: 131ms	remaining: 2m 10s
100:	total: 3.22s	remaining: 28.7s
200:	total: 5.81s	remaining: 23.1s
300:	total: 8.31s	remaining: 19.3s
400:	total: 10.7s	remaining: 16s
500:	total: 13.2s	remaining: 13.2s
600:	total: 15.7s	remaining: 10.4s
700:	total: 18.4s	remaining: 7.86s
800:	total: 21.3s	remaining: 5.29s
900:	total: 23.9s	remaining: 2.62s
999:	total: 26.6s	remaining: 0us


In [20]:
# Predict the labels for the val set
y_pred = model_catb.predict(X_val)
y_probs = model_catb.predict_proba(X_val)

# Generate the confusion matrix
cm = confusion_matrix(y_val, y_pred)

# Generate classification report
report = classification_report(y_val, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.88      0.87      0.88       250
           1       0.91      0.92      0.91       345

    accuracy                           0.90       595
   macro avg       0.90      0.89      0.89       595
weighted avg       0.90      0.90      0.90       595



testing the model on the test set to ensure the accuracy is the same before and after conversion to ONNX

In [24]:
test_df = df_test.to_pandas()
test_df.head()

,tid,cleaned_doc,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,Sentiment
0,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1.582610,1.252096,1.246331,-2.210566,-0.014998,0.320480,-2.154431,-1.159533,...,0.823797,0.896441,-0.089060,-0.165477,-0.669874,0.585659,0.447458,-0.787317,-0.296942,1
1,10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وه...,0.119550,0.641493,1.329657,-1.626576,-0.244940,0.976920,-2.952441,0.074009,...,0.202292,0.983413,-0.280799,-1.034488,2.325436,0.122212,0.485375,0.694378,-1.464473,1
2,7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المت...,-0.639943,-1.222346,1.922086,-1.380060,0.203868,-0.311772,-2.933300,0.364731,...,0.261476,0.724736,-1.346588,-0.735644,2.314811,0.762845,0.567371,1.422184,-1.089902,1
3,6,اومن حقوق المرأة أنك تفتحلها الباب,-0.815878,-0.573813,0.875430,0.729033,-1.369993,-0.599379,-2.230564,0.245553,...,0.136895,0.830590,-0.223158,0.011948,1.432901,-0.107069,1.015441,1.372740,0.151745,1
4,8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,-0.694978,-0.020998,1.735893,-0.455287,-0.197177,0.546692,-2.294147,0.101428,...,-0.452604,0.351147,-0.382032,-0.814823,1.567826,0.536734,1.648678,-1.004110,-0.132113,1


In [22]:
testdep = test_df[[f"emb_{i}" for i in range(768)]].values
testlab = test_df["Sentiment"].values
testdep.shape, testlab.shape

((1276, 768), (1276,))

In [23]:
testpred = model_catb.predict(testdep)
testproba = model_catb.predict_proba(testdep)

# Metrics
acc_catboost = accuracy_score(testlab, testpred)
f1_catboost = f1_score(testlab, testpred)
auc_catboost = roc_auc_score(testlab, testproba[:, 1])
print(f"CatBoost - Test Set Accuracy: {acc_catboost*100:.2f}%, F1 Score: {f1_catboost*100:.2f}%, AUC: {auc_catboost*100:.2f}%")

CatBoost - Test Set Accuracy: 88.87%, F1 Score: 89.99%, AUC: 96.23%


So the accuracy of the model before conversion is 87.23%

## 6. Converting the Model to ONNX

Converting the model to ONNX file only

In [25]:
onnxfile_catb = "catb-sentiment.onnx"
model_catb_id = "catb_sentiment"

model_catb.save_model(onnxfile_catb,
                 format="onnx",
    export_parameters={
        'onnx_domain': 'ai.catboost',
        'onnx_model_version': 1,
        'onnx_doc_string': 'model for sentiment classification of arabic tweets',
        'onnx_graph_name': 'CatBoostModel_for_SentimentClassification'
    })

In [30]:
sess = ort.InferenceSession(onnxfile_catb)

label, probabilities = sess.run(['label', 'probabilities'],
                                {'features': testdep.astype(np.float32)})
print(probabilities)

[{0: 0.034287452697753906, 1: 0.9657125473022461}, {0: 0.8297757506370544, 1: 0.17022424936294556}, {0: 0.12145328521728516, 1: 0.8785467147827148}, {0: 0.017223596572875977, 1: 0.982776403427124}, {0: 0.33486974239349365, 1: 0.6651302576065063}, {0: 0.9767516851425171, 1: 0.02324831485748291}, {0: 0.008387207984924316, 1: 0.9916127920150757}, {0: 0.046980977058410645, 1: 0.9530190229415894}, {0: 0.7556836605072021, 1: 0.24431633949279785}, {0: 0.040869951248168945, 1: 0.959130048751831}, {0: 0.0442240834236145, 1: 0.9557759165763855}, {0: 0.1611461043357849, 1: 0.8388538956642151}, {0: 0.12788313627243042, 1: 0.8721168637275696}, {0: 0.06754451990127563, 1: 0.9324554800987244}, {0: 0.11462140083312988, 1: 0.8853785991668701}, {0: 0.7455363273620605, 1: 0.25446367263793945}, {0: 0.002736687660217285, 1: 0.9972633123397827}, {0: 0.1563018560409546, 1: 0.8436981439590454}, {0: 0.01823413372039795, 1: 0.981765866279602}, {0: 0.018980443477630615, 1: 0.9810195565223694}, {0: 0.230123639106

In [31]:
label.shape

(1276,)

In [32]:
prob_class1 = np.array([prob[1] for prob in probabilities])
print(prob_class1)

# Predict the labels for the test set
acc_catboost = accuracy_score(testlab, label)
f1_catboost = f1_score(testlab, label)
auc_catboost = roc_auc_score(testlab, prob_class1)
print(f"CatBoost - ONNX Test Set Accuracy: {acc_catboost*100:.2f}%, F1 Score: {f1_catboost*100:.2f}%, AUC: {auc_catboost*100:.2f}%")

[9.65712547e-01 1.70224249e-01 8.78546715e-01 ... 1.20489001e-02
 1.73729658e-03 8.79466534e-04]
CatBoost - ONNX Test Set Accuracy: 88.87%, F1 Score: 89.99%, AUC: 96.23%


## 7. Loading the model into Vantage

Loading the model into vantage for inference at scale

In [33]:
tdml.configure.byom_install_location = "mldb"
model_table_name = "arabic_sentiment_classifier"

try:
    tdml.db_drop_table(model_table_name)
except:
    pass

tdml.save_byom(model_id = model_catb_id, 
               model_file =  onnxfile_catb, 
               table_name =  model_table_name)

Created the model table 'arabic_sentiment_classifier' as it does not exist.
Model is saved.


In [34]:
query = f"SELECT * FROM {model_table_name};"
tdml.DataFrame.from_query(query)

model_id,model
catb_sentiment,b'8031208436174426F...'


## 8. Scoring using the model in Vantage

Time to infer test dataset through the classifier inside Vantage

In [35]:
onnxpred_obj = tdml.ONNXPredict(
    newdata = df_test,
    modeldata = tdml.DataFrame(model_table_name),
    accumulate = ["tid", "Sentiment"],
    overwrite_cached_models = "*",
    model_input_fields_map = f"features=emb_0:emb_{768-1}" 
)

print(onnxpred_obj.show_query())

SELECT * FROM "mldb".ONNXPredict(
	ON "DEMO_USER"."ml__select__1761524088148939" AS InputTable
	PARTITION BY ANY 
	ON "arabic_sentiment_classifier" AS ModelTable
	DIMENSION
	USING
	Accumulate('tid', 'Sentiment')
	OverwriteCachedModel('*')
	ModelInputFieldsMap('features=emb_0:emb_767')
) as sqlmr


In [36]:
onnxpred_obj.result

tid,Sentiment,json_report
32,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.04086995,""1"":0.95913005},""type"":""ONNX_TYPE_MAP""}]}"
27,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.008387208,""1"":0.9916128},""type"":""ONNX_TYPE_MAP""}]}"
39,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.044224203,""1"":0.9557758},""type"":""ONNX_TYPE_MAP""}]}"
6,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.017223597,""1"":0.9827764},""type"":""ONNX_TYPE_MAP""}]}"
29,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.1611461,""1"":0.8388539},""type"":""ONNX_TYPE_MAP""}]}"
2,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.034287453,""1"":0.96571255},""type"":""ONNX_TYPE_MAP""}]}"
8,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.33486974,""1"":0.66513026},""type"":""ONNX_TYPE_MAP""}]}"
24,1,"{""label"":[0],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.75568366,""1"":0.24431634},""type"":""ONNX_TYPE_MAP""}]}"
19,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.046980858,""1"":0.95301914},""type"":""ONNX_TYPE_MAP""}]}"
7,1,"{""label"":[1],""probabilities"":[{""info"":{""size"":2,""keyType"":""INT64"",""valueType"":""FLOAT""},""value"":{""0"":0.121453226,""1"":0.8785468},""type"":""ONNX_TYPE_MAP""}]}"


In [37]:
import json
result = onnxpred_obj.result.to_pandas()


# Parse the JSON column
def extract_predicted_label(json_str):
    result = json.loads(json_str)
    return result['label'][0]

result['predicted'] = result['json_report'].apply(extract_predicted_label)

y_test = result['Sentiment'].values
y_pred = result['predicted'].values

# Calculate accuracy
acc_catboost = accuracy_score(y_test, y_pred)
f1_catboost = f1_score(y_test, y_pred)

print(f"CatBoost - Test Set Accuracy in Vantage: {acc_catboost*100:.2f}%, F1 Score: {f1_catboost*100:.2f}%")

CatBoost - Test Set Accuracy in Vantage: 88.87%, F1 Score: 89.99%


# Clean-Up

In [38]:
tables_query = """
SELECT TableName 
FROM DBC.TablesV 
WHERE DatabaseName = 'demo_user' 
AND TableKind = 'T'
"""

tables_df = tdml.DataFrame.from_query(tables_query).to_pandas()

# Drop each table
for table_name in tables_df['TableName']:
    try:
        tdml.db_drop_table(table_name)
        print(f"✓ Dropped table: {table_name}")
    except Exception as e:
        print(f"✗ Failed to drop {table_name}: {e}")

print("All tables dropped!")

tdml.remove_context()

✓ Dropped table: arabic_sentiment_classifier
✓ Dropped table: arabic_sentiment_data
All tables dropped!


True